In [1]:
# !pip install tensorflow
# !pip install keras

In [60]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
import random
from tensorflow import keras
import numpy as np
# from tensorflow.keras import layers, Dense, Input, InputLayer, Flatten
# from tensorflow.keras.models import Sequential, Model

from keras import layers
from keras.layers import Dense,Input, InputLayer, Flatten


from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [61]:
# img_folder=r'/content/drive/MyDrive/Major Project/Full Dataset'
img_folder = r'./Datasets/Test/'

In [62]:
# image = cv2.imread(r'./Datasets/Test/bright_dune/ESP_016576_2570_RED-0003.jpg')
# image = np.array(image)
# image = image.astype('float32')
# image /= 255
# print(image.shape)

In [ ]:
def create_dataset(img_folder):

    img_data_array = list()
    class_name=[]
    
    for dir1 in os.listdir(img_folder):
        for file in os.listdir(os.path.join(img_folder, dir1)):
            image_path= os.path.join(img_folder, dir1,  file)
            print(image_path)
            image = cv2.imread(image_path)
            image = np.array(image)
            image = image.astype('float32')
            image /= 255 
            img_data_array.append(image)
            class_name.append(dir1)
            print(len(class_name))
    return img_data_array, class_name

# extract the image array and class name
img_data, class_name = create_dataset(img_folder)

In [64]:
target_dict={k: v for v, k in enumerate(np.unique(class_name))}
target_dict

{'bright_dune': 0, 'crater': 1, 'dark_dune': 2, 'streak': 3}

In [65]:
target_val =  [target_dict[class_name[i]] for i in range(len(class_name))]

In [66]:
print(len(target_val), len(img_data))

970 970


In [67]:
model=tf.keras.Sequential(
        [
            tf.keras.layers.InputLayer(input_shape=(227,227, 3)),
            tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=(2, 2), activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(5)
        ])

In [68]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [69]:
history = model.fit(x=np.array(img_data, np.float32), y=np.array(list(map(int,target_val)), np.float32), epochs=15)

Train on 970 samples
Epoch 1/15
970/970 [==============================] - 2s 2ms/sample - loss: 9.4285 - accuracy: 0.4082
Epoch 2/15
970/970 [==============================] - 1s 857us/sample - loss: 9.3385 - accuracy: 0.4206
Epoch 3/15
970/970 [==============================] - 1s 858us/sample - loss: 9.3385 - accuracy: 0.4206
Epoch 4/15
970/970 [==============================] - 1s 872us/sample - loss: 9.3385 - accuracy: 0.4206
Epoch 5/15
970/970 [==============================] - 1s 868us/sample - loss: 9.3385 - accuracy: 0.4206
Epoch 6/15
970/970 [==============================] - 1s 878us/sample - loss: 9.3385 - accuracy: 0.4206
Epoch 7/15
970/970 [==============================] - 1s 854us/sample - loss: 9.3385 - accuracy: 0.4206
Epoch 8/15
970/970 [==============================] - 1s 854us/sample - loss: 9.3385 - accuracy: 0.4206
Epoch 9/15
970/970 [==============================] - 1s 876us/sample - loss: 9.3385 - accuracy: 0.4206
Epoch 10/15
970/970 [========================

In [70]:
def findClass(image_path):
    image_1 = cv2.imread(image_path)
    image_1 = np.array(image_1)
    image_1 = image_1.astype('float32')
    image_1 /= 255 
    prediction = model.predict(np.array([image_1], np.float32))
    print(prediction)
    return np.argmin(prediction)

In [74]:
findClass(r'./Datasets/Test/bright_dune/ESP_016576_2570_RED-0061.jpg')
# findClass(r'./Datasets/Test/dark_dune/ESP_018334_2600_RED-0501.jpg')
# findClass(r'./Datasets/Test/streak/ESP_011289_1950_RED-0067.jpg')
# {'bright_dune': 0, 'crater': 1, 'dark_dune': 2, 'other': 3, 'streak': 4}

[[-47.79378  -47.461697  44.052372 -49.08728  -49.16688 ]]


4